In [1]:
from IPython.display import display
from IPython.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import hail as hl
import sys
import os

hl.init(tmp_dir=os.path.join(os.environ.get('SCRATCH'), 'hail-tmpdir'),
 local_tmpdir=os.path.join(os.environ.get('SCRATCH_LOCAL'), 'hail-local-tmpdir'),
 spark_conf={'spark.driver.memory': '20G', 'spark.executor.memory': '20G'}, default_reference='GRCh38') 

2022-04-25 10:05:58 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.2
SparkUI available at http://p0577.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.85-9b98676b6ad8
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/analysis/hail-20220425-1005-0.2.85-9b98676b6ad8.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis
from scipy import stats
import seaborn as sns

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [4]:
controls = [ 'S_7212','S_7213','S_7227','S_7255','S_7237','S_7245','S_7246','S_7229','S_7254','WGS_147c','S_7261','S_7263','S_7269','S_7274','S_7294','S_7306','WGS_37b','WGS_37c','WGS_85b','WGS_7118',
'WGS_7120','WGS_7142','WGS_7143','WGS_7152','WGS_7153','WGS_163d','WGS_180b','WGS_6819','WGS_D6813','WGS_D6815','462','468','475','476','477','478','479','482','490','492','494']

In [5]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/dense-all.mt')

In [62]:
mt = hl.filter_intervals(mt, [(hl.parse_locus_interval('chr2:190055699-190062728', reference_genome='GRCh38'))])

In [63]:
mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'GTS'))))

In [64]:
mt = mt.filter_cols(
    (mt.group == 'sport') |
    (hl.literal(controls).contains(mt.s))
    )

In [65]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [66]:
mt.count()

(55, 143)

In [67]:
mt = mt.annotate_rows(dp_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.DP)),
                      gq_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)),
                      hwe = hl.agg.group_by(mt.group, hl.agg.hardy_weinberg_test(mt.GT)))

mt = mt.annotate_rows(n_below_dp_3 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.DP < 3)),
                      n_below_gq_30 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GQ <30)))

In [68]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/to-delete.mt')

2022-04-13 19:04:38 Hail: INFO: wrote matrix table with 55 rows and 143 columns in 1 partition to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/to-delete.mt


In [69]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/to-delete.mt')

In [70]:
mt = mt.filter_rows((mt.dp_qc.get('GTS', hl.struct(mean=6.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.dp_qc.get('sport', hl.struct(mean=6.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.gq_qc.get('GTS', hl.struct(mean=60.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.gq_qc.get('sport', hl.struct(mean=60.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.n_below_dp_3.get('sport', 0) < 3) &
                    (mt.n_below_gq_30.get('sport', 0) < 30) &
                    (mt.n_below_dp_3.get('GTS', 0) < 3) &
                    (mt.n_below_gq_30.get('GTS', 0) <30))

In [72]:
mt.count()

(53, 143)

In [80]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/resources/repeatmasker-extended-keyed.ht')
cov = hl.read_table('/net/archive/groups/plggneuromol/resources/gnomad/gnomad.genomes.r3.0.1.coverage.ht')

In [81]:
cov = cov.filter(cov.over_1 > 0.9)

In [82]:
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)

In [83]:
mt.count()

(40, 143)

In [84]:
mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)

In [85]:
mt.count()

(40, 143)

In [88]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/myostatin.mt')

2022-04-13 19:12:55 Hail: INFO: wrote matrix table with 40 rows and 143 columns in 1 partition to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/myostatin.mt
    Total size: 27.87 KiB
    * Rows/entries: 27.22 KiB
    * Columns: 651.00 B
    * Globals: 11.00 B
    * Smallest partition: 40 rows (27.22 KiB)
    * Largest partition:  40 rows (27.22 KiB)


In [95]:
genes = hl.read_table('/net/archive/groups/plggneuromol/resources/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))
hpo = hl.import_table('/net/archive/groups/plggneuromol/resources/hpo.tsv', impute = True, no_header=True)

2022-04-13 19:28:05 Hail: INFO: Reading table to impute column types
2022-04-13 19:28:06 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type str (imputed)


In [96]:
start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])
genes = genes.key_by(genes.interval)

gnomad = hl.read_table('/net/archive/groups/plggneuromol/resources/gnomad/gnomad.genomes.v3.1.1.sites.ht/')
cadd = hl.read_table('/net/archive/groups/plggneuromol/resources/cadd.ht')
vep = hl.read_table('/net/archive/groups/plggneuromol/resources/vep38/grch38_context_vep_annotated.ht')
sport_pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sportsmen-pheno.csv', impute = True, key='sample_id', quote ="\"")
poles = hl.read_matrix_table('/net/archive/groups/plggneuromol/resources/polish-genomes/polish-genomes.mt')

2022-04-13 19:28:08 Hail: INFO: Reading table to impute column types
2022-04-13 19:28:09 Hail: INFO: Finished type imputation
  Loading field 'sport' as type str (imputed)
  Loading field 'type' as type str (imputed)
  Loading field 'age' as type int32 (imputed)
  Loading field 'sample_id' as type str (imputed)


In [ ]:
mt = mt.distinct_by_row()
mt = mt.key_rows_by(mt.locus, mt.alleles)
mt = hl.split_multi_hts(mt)
mt = mt.annotate_rows(gnomad_v_3_1 = gnomad[mt.row_key])
mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
mt = mt.annotate_rows(cadd = cadd[mt.row_key])
mt = mt.annotate_rows(vep = vep[mt.row_key])
mt = mt.annotate_cols(sport_phenotypes = sport_pheno[mt.s])
mt = mt.annotate_rows(polish_af = poles.rows()[mt.row_key]['info'])


mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/mstn.mt')

In [102]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/mstn.mt')

In [103]:
mt.count()

(41, 143)

In [104]:
mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'control'))))

mt = mt.annotate_rows(het_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_var())))

mt = mt.annotate_rows(het_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_var())))

sport_AC_non_ref = hl.int32(mt.het_non_refs.get('sport', 0) + (mt.hom_non_refs.get('sport', 0)*2))
sport_AC_ref = hl.int32(mt.het_non_refs.get('sport', 0) + (mt.hom_refs.get('sport', 0)*2))
control_AC_non_ref = hl.int32(mt.het_non_refs.get('control', 0) + (mt.hom_non_refs.get('control', 0)*2))
control_AC_ref = hl.int32(mt.het_non_refs.get('control', 0) + (mt.hom_refs.get('control', 0)*2))
speed_AC_non_ref = hl.int32(mt.het_non_refs_e_vs_s.get('speed', 0) + (mt.hom_non_refs_e_vs_s.get('speed', 0)*2))
speed_AC_ref = hl.int32(mt.het_non_refs_e_vs_s.get('speed', 0) + (mt.hom_refs_e_vs_s.get('speed', 0)*2))
endurance_AC_non_ref = hl.int32(mt.het_non_refs_e_vs_s.get('endurance', 0) + (mt.hom_non_refs_e_vs_s.get('endurance', 0)*2))
endurance_AC_ref = hl.int32(mt.het_non_refs_e_vs_s.get('endurance', 0) + (mt.hom_refs_e_vs_s.get('endurance', 0)*2))
gnomad_AC_non_ref = mt.gnomad_v_3_1.freq.AC[2]
gnomad_AC_ref = mt.gnomad_v_3_1.freq.AN[2] - mt.gnomad_v_3_1.freq.AC[2]
polish_AC_non_ref = hl.int32(mt.polish_af.AC[0])
polish_AC_ref = hl.int32(mt.polish_af.AN - mt.polish_af.AC[0])

mt = mt.annotate_rows(fisher_sport_vs_control = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, control_AC_non_ref, control_AC_ref),
                              fisher_e_vs_s = hl.fisher_exact_test(speed_AC_non_ref, speed_AC_ref, endurance_AC_non_ref, endurance_AC_ref),
                              fisher_sport_vs_gnomad = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, gnomad_AC_non_ref, gnomad_AC_ref),
                              fisher_sport_vs_polish = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, polish_AC_non_ref, polish_AC_ref))

mt = mt.drop(mt['a_index'], mt['was_split'])

mt = mt.transmute_rows(DP_stats = mt.dp_qc,
                             GQ_stats = mt.gq_qc, 
                             hwe_controls_p_value = mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
                             hwe_sport_p_value = mt.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
                             gnomad_v3_nfe_af = mt.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = mt.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = mt.cadd.score_phred,
                             within_gene = mt.within_gene,
                             hpo = mt.hpo,
                             fisher_sport_vs_control  = mt.fisher_sport_vs_control,
                             fisher_e_vs_s = mt.fisher_e_vs_s ,
                             fisher_sport_vs_gnomad  = mt.fisher_sport_vs_gnomad,
                             fisher_sport_vs_polish = mt.fisher_sport_vs_polish,                         
                             het_sport = mt.het_non_refs.get('sport', 0),
                             het_controls = mt.het_non_refs.get('control', 0),
                             hom_ref_sport = mt.hom_refs.get('sport', 0),
                             hom_ref_controls = mt.hom_refs.get('control', 0),
                             hom_var_sport = mt.hom_non_refs.get('sport', 0),
                             how_var_controls = mt.hom_non_refs.get('control',0),
                             het_endurance = mt.het_non_refs_e_vs_s.get('endurance', 0),
                             het_speed = mt.het_non_refs_e_vs_s.get('speed', 0),
                             hom_ref_endurance = mt.hom_refs_e_vs_s.get('endurance', 0),
                             hom_ref_speed = mt.hom_refs_e_vs_s.get('speed', 0),
                             hom_var_endurance = mt.hom_non_refs_e_vs_s.get('endurance', 0),
                             how_var_speed = mt.hom_non_refs_e_vs_s.get('speed',0),
                             most_severe_consequence = mt.vep.vep.most_severe_consequence,
                             transcript_consequences = mt.vep.vep.transcript_consequences,
                             intergenic_consequences = mt.vep.vep.intergenic_consequences,
                             motif_feature_consequences = mt.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = mt.vep.vep.regulatory_feature_consequences,
                             polish_af = mt.polish_af)

mt = mt.select_entries(mt.GT)

mt = mt.make_table()
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/mstn-to-export.mt')

2022-04-13 19:34:33 Hail: INFO: wrote table with 41 rows in 1 partition to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/mstn-to-export.mt


In [106]:
mt = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/mstn-to-export.mt')

to_export = mt.flatten()
to_export.export('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/myostatin.csv')
to_export.export('myostatin.csv')

2022-04-13 19:34:56 Hail: INFO: merging 1 files totalling 172.3K...
2022-04-13 19:34:56 Hail: INFO: while writing:
    /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/myostatin.csv
  merge time: 15.716ms
2022-04-13 19:34:57 Hail: INFO: merging 1 files totalling 172.3K...
2022-04-13 19:34:57 Hail: INFO: while writing:
    myostatin.csv
  merge time: 11.019ms


#### Export of two other genes:

#### ACVR2A i ACVR2B

    x chr2:147,844,517-147,930,826
    chr3:38,453,890-38,493,142

In [6]:
intervals = ['chr2:147844517-147930826', 'chr3:38453890-38493142']

mt = hl.filter_intervals(
    mt,
    [hl.parse_locus_interval(x, reference_genome='GRCh38') for x in intervals]
)

In [7]:
mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'GTS'))))

In [8]:
mt = mt.filter_cols(
    (mt.group == 'sport') |
    (hl.literal(controls).contains(mt.s))
    )

In [9]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [10]:
mt.count()

(768, 143)

In [11]:
mt = mt.annotate_rows(dp_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.DP)),
                      gq_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)),
                      hwe = hl.agg.group_by(mt.group, hl.agg.hardy_weinberg_test(mt.GT)))

mt = mt.annotate_rows(n_below_dp_3 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.DP < 3)),
                      n_below_gq_30 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GQ <30)))

In [12]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/to-delete.mt')

2022-04-25 10:11:02 Hail: INFO: wrote matrix table with 768 rows and 143 columns in 2 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/to-delete.mt


In [13]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/to-delete.mt')

In [14]:
mt = mt.filter_rows((mt.dp_qc.get('GTS', hl.struct(mean=6.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.dp_qc.get('sport', hl.struct(mean=6.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.gq_qc.get('GTS', hl.struct(mean=60.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.gq_qc.get('sport', hl.struct(mean=60.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.n_below_dp_3.get('sport', 0) < 3) &
                    (mt.n_below_gq_30.get('sport', 0) < 30) &
                    (mt.n_below_dp_3.get('GTS', 0) < 3) &
                    (mt.n_below_gq_30.get('GTS', 0) <30))

In [15]:
mt.count()

(550, 143)

In [16]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/resources/repeatmasker-extended-keyed.ht')
cov = hl.read_table('/net/archive/groups/plggneuromol/resources/gnomad/gnomad.genomes.r3.0.1.coverage.ht')

In [17]:
cov = cov.filter(cov.over_1 > 0.9)

In [18]:
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)

In [19]:
mt.count()

(361, 143)

In [20]:
mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)

In [21]:
mt.count()

(361, 143)

In [22]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/acvr.mt')

2022-04-25 10:12:11 Hail: INFO: wrote matrix table with 361 rows and 143 columns in 2 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/acvr.mt
    Total size: 344.87 KiB
    * Rows/entries: 344.22 KiB
    * Columns: 651.00 B
    * Globals: 11.00 B
    * Smallest partition: 151 rows (156.65 KiB)
    * Largest partition:  210 rows (187.57 KiB)


In [23]:
genes = hl.read_table('/net/archive/groups/plggneuromol/resources/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))
hpo = hl.import_table('/net/archive/groups/plggneuromol/resources/hpo.tsv', impute = True, no_header=True)

2022-04-25 10:12:11 Hail: INFO: Reading table to impute column types
2022-04-25 10:12:12 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type str (imputed)


In [24]:
start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])
genes = genes.key_by(genes.interval)

gnomad = hl.read_table('/net/archive/groups/plggneuromol/resources/gnomad/gnomad.genomes.v3.1.1.sites.ht/')
cadd = hl.read_table('/net/archive/groups/plggneuromol/resources/cadd.ht')
vep = hl.read_table('/net/archive/groups/plggneuromol/resources/vep38/grch38_context_vep_annotated.ht')
sport_pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sportsmen-pheno.csv', impute = True, key='sample_id', quote ="\"")
poles = hl.read_matrix_table('/net/archive/groups/plggneuromol/resources/polish-genomes/polish-genomes.mt')

2022-04-25 10:12:14 Hail: INFO: Reading table to impute column types
2022-04-25 10:12:15 Hail: INFO: Finished type imputation            (0 + 1) / 1]
  Loading field 'sport' as type str (imputed)
  Loading field 'type' as type str (imputed)
  Loading field 'age' as type int32 (imputed)
  Loading field 'sample_id' as type str (imputed)


In [25]:
mt = mt.distinct_by_row()
mt = mt.key_rows_by(mt.locus, mt.alleles)
mt = hl.split_multi_hts(mt)
mt = mt.annotate_rows(gnomad_v_3_1 = gnomad[mt.row_key])
mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
mt = mt.annotate_rows(cadd = cadd[mt.row_key])
mt = mt.annotate_rows(vep = vep[mt.row_key])
mt = mt.annotate_cols(sport_phenotypes = sport_pheno[mt.s])
mt = mt.annotate_rows(polish_af = poles.rows()[mt.row_key]['info'])


mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr-anno.mt')

2022-04-25 10:12:36 Hail: INFO: Ordering unsorted dataset with network shuffle2]
2022-04-25 10:12:38 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-04-25 10:12:40 Hail: INFO: Ordering unsorted dataset with network shuffle2]
2022-04-25 10:12:40 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-04-25 10:12:41 Hail: INFO: Ordering unsorted dataset with network shuffle1]
2022-04-25 10:12:42 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-04-25 10:13:03 Hail: INFO: wrote matrix table with 402 rows and 143 columns in 2 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr-anno.mt
    Total size: 2.04 MiB
    * Rows/entries: 2.04 MiB
    * Columns: 1.15 KiB
    * Globals: 11.00 B
    * Smallest partition: 166 rows (892.24 KiB)
    * Largest partition:  236 rows (1.17 MiB)


In [26]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr-anno.mt')

In [27]:
mt.count()

(402, 143)

In [28]:
mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'control'))))

mt = mt.annotate_rows(het_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_var())))

mt = mt.annotate_rows(het_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_var())))

sport_AC_non_ref = hl.int32(mt.het_non_refs.get('sport', 0) + (mt.hom_non_refs.get('sport', 0)*2))
sport_AC_ref = hl.int32(mt.het_non_refs.get('sport', 0) + (mt.hom_refs.get('sport', 0)*2))
control_AC_non_ref = hl.int32(mt.het_non_refs.get('control', 0) + (mt.hom_non_refs.get('control', 0)*2))
control_AC_ref = hl.int32(mt.het_non_refs.get('control', 0) + (mt.hom_refs.get('control', 0)*2))
speed_AC_non_ref = hl.int32(mt.het_non_refs_e_vs_s.get('speed', 0) + (mt.hom_non_refs_e_vs_s.get('speed', 0)*2))
speed_AC_ref = hl.int32(mt.het_non_refs_e_vs_s.get('speed', 0) + (mt.hom_refs_e_vs_s.get('speed', 0)*2))
endurance_AC_non_ref = hl.int32(mt.het_non_refs_e_vs_s.get('endurance', 0) + (mt.hom_non_refs_e_vs_s.get('endurance', 0)*2))
endurance_AC_ref = hl.int32(mt.het_non_refs_e_vs_s.get('endurance', 0) + (mt.hom_refs_e_vs_s.get('endurance', 0)*2))
gnomad_AC_non_ref = mt.gnomad_v_3_1.freq.AC[2]
gnomad_AC_ref = mt.gnomad_v_3_1.freq.AN[2] - mt.gnomad_v_3_1.freq.AC[2]
polish_AC_non_ref = hl.int32(mt.polish_af.AC[0])
polish_AC_ref = hl.int32(mt.polish_af.AN - mt.polish_af.AC[0])

mt = mt.annotate_rows(fisher_sport_vs_control = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, control_AC_non_ref, control_AC_ref),
                              fisher_e_vs_s = hl.fisher_exact_test(speed_AC_non_ref, speed_AC_ref, endurance_AC_non_ref, endurance_AC_ref),
                              fisher_sport_vs_gnomad = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, gnomad_AC_non_ref, gnomad_AC_ref),
                              fisher_sport_vs_polish = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, polish_AC_non_ref, polish_AC_ref))

mt = mt.drop(mt['a_index'], mt['was_split'])

mt = mt.transmute_rows(DP_stats = mt.dp_qc,
                             GQ_stats = mt.gq_qc, 
                             hwe_controls_p_value = mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
                             hwe_sport_p_value = mt.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
                             gnomad_v3_nfe_af = mt.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = mt.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = mt.cadd.score_phred,
                             within_gene = mt.within_gene,
                             hpo = mt.hpo,
                             fisher_sport_vs_control  = mt.fisher_sport_vs_control,
                             fisher_e_vs_s = mt.fisher_e_vs_s ,
                             fisher_sport_vs_gnomad  = mt.fisher_sport_vs_gnomad,
                             fisher_sport_vs_polish = mt.fisher_sport_vs_polish,                         
                             het_sport = mt.het_non_refs.get('sport', 0),
                             het_controls = mt.het_non_refs.get('control', 0),
                             hom_ref_sport = mt.hom_refs.get('sport', 0),
                             hom_ref_controls = mt.hom_refs.get('control', 0),
                             hom_var_sport = mt.hom_non_refs.get('sport', 0),
                             how_var_controls = mt.hom_non_refs.get('control',0),
                             het_endurance = mt.het_non_refs_e_vs_s.get('endurance', 0),
                             het_speed = mt.het_non_refs_e_vs_s.get('speed', 0),
                             hom_ref_endurance = mt.hom_refs_e_vs_s.get('endurance', 0),
                             hom_ref_speed = mt.hom_refs_e_vs_s.get('speed', 0),
                             hom_var_endurance = mt.hom_non_refs_e_vs_s.get('endurance', 0),
                             how_var_speed = mt.hom_non_refs_e_vs_s.get('speed',0),
                             most_severe_consequence = mt.vep.vep.most_severe_consequence,
                             transcript_consequences = mt.vep.vep.transcript_consequences,
                             intergenic_consequences = mt.vep.vep.intergenic_consequences,
                             motif_feature_consequences = mt.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = mt.vep.vep.regulatory_feature_consequences,
                             polish_af = mt.polish_af)

mt = mt.select_entries(mt.GT)

mt = mt.make_table()
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr-to-export.mt')

2022-04-25 10:13:15 Hail: INFO: wrote table with 402 rows in 2 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr-to-export.mt


In [29]:
mt = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr-to-export.mt')

to_export = mt.flatten()
to_export.export('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr.csv')
to_export.export('acvr.csv')

2022-04-25 10:13:34 Hail: INFO: merging 2 files totalling 2.0M...
2022-04-25 10:13:34 Hail: INFO: while writing:
    /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/hail-mts/acvr.csv
  merge time: 74.203ms
2022-04-25 10:13:34 Hail: INFO: merging 2 files totalling 2.0M...
2022-04-25 10:13:34 Hail: INFO: while writing:
    acvr.csv
  merge time: 16.756ms
